In [ ]:
import cv2
import numpy as np
import os
from google.colab import drive
import shutil
import glob
import kagglehub

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Download Dataset 3
path = kagglehub.dataset_download("ashfakyeafi/road-vehicle-images-dataset")

print("Path to dataset files:", path)

!mv /root/.cache/kagglehub/datasets/ashfa1kyeafi/road-vehicle-images-dataset/versions/2/trafic_data /content/drive/MyDrive/Project_CPV303_Group2/unprocess_Dataset


100%|██████████| 115M/115M [00:00<00:00, 185MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/ashfakyeafi/road-vehicle-images-dataset/versions/2


In [ ]:
### Tải DATASET 10k ###
!pip install roboflow --quiet
from roboflow import Roboflow

rf = Roboflow(api_key="nideObRNlv0S8PquBugw")
project = rf.workspace("cv-2022-kyjj6").project("tesi")
version = project.version(2)
out_dir = "/content/drive/MyDrive/"
dataset = version.download("yolov8", location=out_dir)

print(f"Đã tải xong dataset vào: {out_dir}")
print(len(os.listdir('/content/drive/MyDrive/Project_CPV303_Group2/unprocess_Dataset/tesi_v2_1/test/images')))

In [ ]:
!pip install roboflow --quiet
from roboflow import Roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 83.5 MB/s eta 0:00:00


In [ ]:
### tải DATASET 4k ###


rf = Roboflow(api_key="nideObRNlv0S8PquBugw")
project = rf.workspace("roboflow-100").project("vehicles-q0x2v")
version = project.version(1)
outdir_= "/content/drive/MyDrive/"
dataset = version.download("yolov11", location= outdir_, overwrite=True)

print(f"Đã tải xong dataset vào: {outdir_}")
print(len(os.listdir('/content/drive/MyDrive/Project_CPV303_Group2/unprocess_Dataset/')))

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to vehicles-1 in yolov11:: 100%|██████████| 8128/8128 [00:01<00:00, 5972.09it/s]

Đã tải xong dataset vào: /content/drive/MyDrive/
2


In [ ]:
# Nếu ko tải được vào outdir_ thì tại vào content/ và mv vào thư mục
!mv /content/vehicles-1 /content/drive/MyDrive/Project_CPV303_Group2/unprocess_Dataset

In [ ]:
# Check len của tổng dataset mới vừa tải coi có khớp với trên nguồn ko
print(len(os.listdir('/content/drive/MyDrive/Project_CPV303_Group2/unprocess_Dataset/Dataset_4k_640_processing/train/images'))
+len(os.listdir('/content/drive/MyDrive/Project_CPV303_Group2/unprocess_Dataset/Dataset_4k_640_processing/valid/images'))
+ len(os.listdir('/content/drive/MyDrive/Project_CPV303_Group2/unprocess_Dataset/Dataset_4k_640_processing/test/images')))

4058


In [ ]:
!unzip /content/drive/MyDrive/Project_CPV303_Group2/unprocess_Dataset/tesi.v3i.yolov8.zip -d /content/drive/MyDrive/Project_CPV303_Group2/unprocess_Dataset/

In [ ]:
def is_blurry(image_path, threshold=100):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return True
    return cv2.Laplacian(img, cv2.CV_64F).var() < threshold

def filter_images_label_files(label_path, imgs_path, save_path= None, remove_ = False):

  label_names =set(os.path.splitext(la)[0] for la in os.listdir(label_path) if la.endswith(('.txt')))
  img_files= os.listdir(imgs_path)
  print("image truoc khi loc",len(img_files))
  print("labels",len(label_names))

  if save_path and not os.path.exists(save_path):
    os.makedirs(save_path)

  c= 0
  for name_img in img_files:
    name_p= os.path.splitext(name_img)[0]
    img_path = os.path.join(imgs_path, name_img)

    if is_blurry(img_path):
      os.remove(os.path.join(label_path, name_p + '.txt'))
      if remove_:
        os.remove(os.path.join(imgs_path, name_img))
      continue

    if name_p in label_names and not remove_ :
      shutil.copy(img_path, os.path.join(save_path, name_img))
      c +=1
    elif remove_:
      print('r')
      os.remove(img_path)
  print("image sau khi loc",c)
  print("labels sau khi loc vo Images", len(os.listdir(label_path)))


def fil_labels_no_need(label_path):
  print("labels truoc khi loc",len(os.listdir(label_path)))
  path_files_label = glob.glob(os.path.join(label_path, '*.txt'))
  for p in path_files_label:
    with open(p, 'r') as f:
      lines = f.readlines()
      if len(lines) != 0:
        for l in lines:
          class_= l.strip().split()[0]
          if int(class_) in [1, 2,3] :
            os.remove(p)
            break
      else:
        os.remove(p)
  print("labels sau khi loc",len(os.listdir(label_path)))


def combine_class_dataset(data_path, dict_class_combine):
  label_paths = glob.glob(os.path.join(data_path, '*.txt'))
  class_map = {}
  for new_class, old_classes in dict_class_combine.items():
      for old in old_classes:
          class_map[old] = new_class

  for p in label_paths:
    new_lines = []
    with open(p, 'r') as f:
      lines = f.readlines()
      for l in lines:
        parts = l.strip().split()
        cls = parts[0]
        if cls in class_map:
          parts[0] = class_map[cls]
          new_lines.append(' '.join(parts) + '\n')
    if new_lines:
      with open(p,'w') as fw:
        fw.writelines(new_lines)
        # print('ok')

In [ ]:
# Loc Labels
# Loc Anh theo Label , Neu Anh mo thi xoa hoac bo qua , label thi xoa . Neu ko thi copy qua thu muc Anh moi

In [ ]:
# Dataset 1
images_path = '/content/drive/MyDrive/Project_CPV303_Group2/unprocess_Dataset/tesiv2_10k_640/train/images'
labels_path = '/content/drive/MyDrive/Project_CPV303_Group2/unprocess_Dataset/tesiv2_10k_640/train/labels'

save_labels_path = '/content/drive/MyDrive/Project_CPV303_Group2/unprocess_Dataset/tesiv2_10k_640/train/1_train_labels'
# os.makedirs(save_labels_path, exist_ok=True)

# Dataset 2
label_path_2_train = '/content/drive/MyDrive/Project_CPV303_Group2/unprocess_Dataset/Dataset_4k_640_processing/train/labels'
label_path_2_valid = '/content/drive/MyDrive/Project_CPV303_Group2/unprocess_Dataset/Dataset_4k_640_processing/valid/labels'
label_path_2_test = '/content/drive/MyDrive/Project_CPV303_Group2/unprocess_Dataset/Dataset_4k_640_processing/test/labels'

In [ ]:
combine_class = {
    '1': ['1','11', '8'],
    '2': ['5', '7', '9'],
    '3': ['10'],
    '4': ['4'],
    '0': ['0','2', '3', '6']
}
for pa in [label_path_2_train, label_path_2_valid, label_path_2_test]:
  combine_class_dataset(pa, combine_class)
  print(len(os.listdir(pa)))

print(len(os.listdir(label_path_2_train))
+len(os.listdir(label_path_2_valid))
+len(os.listdir(label_path_2_test)))

2634
966
458
4058


In [ ]:
images_2 = '/content/drive/MyDrive/Project_CPV303_Group2/unprocess_Dataset/tesiv2_10k_640/train/train_deleted_images'
os.makedirs(images_2)

In [ ]:
fil_labels_no_need(labels_path)
filter_images_label_files(labels_path, images_path, images_2, False)

In [ ]:
n_images_path = '/content/drive/MyDrive/Project_CPV303_Group2/unprocess_Dataset/Dataset_4k_640_processing/test/images'
n_labels_path= '/content/drive/MyDrive/Project_CPV303_Group2/unprocess_Dataset/Dataset_4k_640_processing/test/labels'

print(len(os.listdir(n_images_path)))
print(len(os.listdir(n_labels_path)))

458
458


In [ ]:
# No RUN CODE
# shutil.rmtree('/content/drive/MyDrive/Project_CPV303_Group2/unprocess_Dataset/tesi.v3i.yolov8/valid/fixed_labels')

In [ ]:
# No RUN CODE
path_dr= '/content/drive/MyDrive'
my_dr= os.listdir(path_dr)

for _, file_ in enumerate(my_dr):
  if file_.endswith(('.txt', '.yaml')):
      # print(idx, file_)
      # print(len(os.listdir(os.path.join(path_dr, file_))))
    pass
    # os.remove(os.path.join(path_dr, file_))